# Transcribe audio files in batch mode as fast as possible

In [3]:
from importlib.metadata import version

In [2]:
version('torch')

'2.4.0'

In [ ]:
pip install --upgrade pip

In [ ]:
pip install --upgrade transformers accelerate

In [4]:
version('transformers')

'4.45.1'

In [5]:
version('accelerate')

'0.34.2'

In [ ]:
pip install --upgrade flash-attn --no-build-isolation

In [8]:
version('flash_attn')

'2.6.3'

## Huggingface Whisper

https://github.com/huggingface/speech-to-speech/blob/main/STT/whisper_stt_handler.py

https://huggingface.co/eustlb/distil-large-v3-fr

In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "eustlb/distil-large-v3-fr"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, 
    use_safetensors=True, low_cpu_mem_usage=True, device_map=device, 
    attn_implementation="flash_attention_2"
)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    return_timestamps=True,
    torch_dtype=torch_dtype
)

# warmup
dummy_input = torch.randn( (1, model.config.num_mel_bins, 3000), dtype=torch_dtype, device=device)
_ = model.generate(dummy_input)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [4]:
# ./audio/2024-09-19 16-32-50.mp3
# - batch size 64, flash attention 2, no compile: 6.14 sec
# - batch size 64, sdpa attention, compile default with fullgraph: 18 sec (first test) / 34 sec (second test)
# => follow line by line the example of https://huggingface.co/eustlb/distil-large-v3-fr, it is the fastest combination

# Test with 2 mp3 files
# - small: 24 min 47 sec (1487 sec), 22.7 MB file
# - big : 1h 24 min 19 sec (5059 sec), 77.2 MB file

# Sequential long-form: pipe(mp3file)
# - gpu memory = 3.1 GB -> 27 sec / 94 sec

# Chunked long-form: pipe(mp3file, chunk_length_s=25, batch_size=xxx)
# batch size 1 : gpu memory = 3.1 GB -> 33 sec / 111 sec
# batch size 8 : gpu memory = 3.6 GB -> 13 sec / 46 sec
# batch size 16 : gpu memory = 4.4 GB -> 11 sec / 37 sec
# batch size 32 : gpu memory = 5.1 GB -> 9.5 sec / 34 sec
# batch size 64 : gpu memory = 7.2 GB -> 10.3 sec / 31 sec
# batch size 128 : gpu memory = 11 GB -> 9.3 sec / 33 sec
# Chunked attention: pipe(mp3file, chunk_length_s=25, batch_size=xxx)

# Sequential long-form algorithm + batch_size 2: pipe([mp3file1, mp3file2], batch_size=2)
# => RuntimeError: The expanded size of the tensor (505922) must match the existing size (148730) at non-singleton dimension 1.  Target sizes: [128, 505922].  Tensor sizes: [128, 148730]

result = pipe("./audio/2024-09-19 16-32-50.mp3")
print(result["text"][:200])

/workspace/wordslab-voice/.venv/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


 Ok, donc dans la première partie, on a fait beaucoup de choses, on s'est posé beaucoup de questions pour pouvoir cadrer, sélectionner, identifier des projets à Basse-Dia qui soient pertinents. La pha


In [6]:
result = pipe("./audio/2024-09-19 15-03-35.mp3")
print(result["text"][:200])

/workspace/wordslab-voice/.venv/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


 Ok. Donc, on poursuit notre énumération de tous les aspects à prendre en compte pour voir si un projet doit être fait, faisable et rentable, et raisonnable en matière d'environnement, etc. Donc, poin


In [ ]:
results = pipe(["./audio/2024-09-19 15-03-35.mp3","./audio/2024-09-19 16-32-50.mp3"], batch_size=2)
for result in results: print(result["text"])

In [7]:
import os
import glob

# Specify the directory containing mp3 files
directory = '/workspace/wordslab-voice/audio'

# Use glob to get all .mp3 files in the directory
mp3_files = glob.glob(os.path.join(directory, '*.mp3'))

# Loop through each mp3 file
for mp3_file in mp3_files:
    # Get the base name of the file (without directory path)
    base_name = os.path.basename(mp3_file)
    
    # Replace the .mp3 extension with .txt to create a new filename
    sequential_txt_file = base_name.replace('.mp3', '_sequential.txt')
    chunked_txt_file = base_name.replace('.mp3', '_chunked.txt')
    
    # Full path of the text file to be written
    sequential_txt_file_path = os.path.join(directory, sequential_txt_file)
    chunked_txt_file_path = os.path.join(directory, chunked_txt_file)

    # Transcribe audio with two methods
    print(f"- {base_name} (sequential) ...")
    sequential_txt = pipe(mp3_file)["text"]
    print("OK")
    
    print(f"- {base_name} (chunked) ...")
    chunked_txt = pipe(mp3_file, chunk_length_s=25, batch_size=32)["text"]
    print("OK")
    
    # Write a text file with the same name as the mp3 file
    with open(sequential_txt_file_path, 'w') as file:
        file.write(sequential_txt)
    print(f"Saved: {sequential_txt_file_path}")
    
    with open(chunked_txt_file_path, 'w') as file:
        file.write(chunked_txt)
    print(f"Saved: {chunked_txt_file_path}")

- 2024-09-24 12-27-09.mp3 (sequential) ...


/workspace/wordslab-voice/.venv/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


OK
- 2024-09-24 12-27-09.mp3 (chuncked) ...


/workspace/wordslab-voice/.venv/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


OK
Saved: /workspace/wordslab-voice/audio/2024-09-24 12-27-09_sequential.txt
Saved: /workspace/wordslab-voice/audio/2024-09-24 12-27-09_chunked.txt
- 2024-09-24 10-46-15.mp3 (sequential) ...


...

OK
Saved: /workspace/wordslab-voice/audio/2024-09-19 15-03-35_sequential.txt
Saved: /workspace/wordslab-voice/audio/2024-09-19 15-03-35_chunked.txt
- 2024-09-19 16-32-50.mp3 (sequential) ...


/workspace/wordslab-voice/.venv/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


OK
- 2024-09-19 16-32-50.mp3 (chuncked) ...


/workspace/wordslab-voice/.venv/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


OK
Saved: /workspace/wordslab-voice/audio/2024-09-19 16-32-50_sequential.txt
Saved: /workspace/wordslab-voice/audio/2024-09-19 16-32-50_chunked.txt


## Formatting

In [9]:
from importlib.metadata import version

In [ ]:
pip install --upgrade vllm

In [10]:
version('vllm')

'0.6.2'

In [1]:
test_models = {                                                                    # OpenLLM leaderboard score
    "llama-3.1" : "meta-llama/Meta-Llama-3.1-8B-Instruct",                         # 100.0 %
    "llama-3.1:w8a16" : "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16",  # 99.8 %    
    "qwen-2.5" : "Qwen/Qwen2.5-7B-Instruct",
    "qwen-2.5:w8a16" : "Qwen/Qwen2.5-7B-Instruct-GPTQ-Int8",
    "qwen-2.5-14b:w8a16" : "Qwen/Qwen2.5-14B-Instruct-GPTQ-Int8",
    "qwen-2.5-14b:w4a16" : "Qwen/Qwen2.5-14B-Instruct-GPTQ-Int4",
    "qwen-2.5-32b:w4a16" : "Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4"
}

In [2]:
from transformers import AutoTokenizer

def format_prompt(messages, model):
    tokenizer = AutoTokenizer.from_pretrained(model)
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

In [3]:
# Authenticate VLLM with Huggingface Hub
import os

with open("/workspace/hftoken", 'r') as file:
    myhftoken = file.read().strip()

os.environ["HF_TOKEN"]=myhftoken

In [4]:
tokenizer = AutoTokenizer.from_pretrained(test_models["llama-3.1"])

In [11]:
import os, glob

directory = '/workspace/wordslab-voice/audio'

text_files = glob.glob(os.path.join(directory, '*.txt'))

textes = []

# Loop through each mp3 file
for text_file in text_files:
    with open(text_file, 'r') as file:
        content = file.read()
        textes.append(content)
        print(f"Read: {text_file} {len(content)} chars => {len(tokenizer(content)['input_ids'])} tokens")

Read: /workspace/wordslab-voice/audio/2024-09-24 11-14-23_chunked.txt 21446 chars => 5581 tokens
Read: /workspace/wordslab-voice/audio/2024-09-24 10-10-14_sequential.txt 32194 chars => 8187 tokens
Read: /workspace/wordslab-voice/audio/2024-09-26 15-35-04_sequential.txt 60032 chars => 15458 tokens
Read: /workspace/wordslab-voice/audio/2024-09-24 10-46-15_chunked.txt 24866 chars => 6266 tokens
Read: /workspace/wordslab-voice/audio/2024-09-24 10-10-14_chunked.txt 32518 chars => 8167 tokens
Read: /workspace/wordslab-voice/audio/2024-09-24 11-39-13_sequential.txt 38849 chars => 9949 tokens
Read: /workspace/wordslab-voice/audio/2024-09-19 14-13-55_chunked.txt 46405 chars => 11883 tokens
Read: /workspace/wordslab-voice/audio/2024-09-19 13-34-26_chunked.txt 33588 chars => 8741 tokens
Read: /workspace/wordslab-voice/audio/2024-09-19 15-03-35_chunked.txt 80679 chars => 20942 tokens
Read: /workspace/wordslab-voice/audio/2024-09-24 11-14-23_sequential.txt 20560 chars => 5263 tokens
Read: /workspac

In [26]:
import time
from vllm import LLM, SamplingParams

def vllm_load(model):    
    llm = LLM(model, gpu_memory_utilization=0.99, max_model_len=49152) # kv_cache_dtype="fp8"
    llm._model = model
    return llm

def vllm_generate(instruction, text, llm):    
    message = instruction + text
    prompt = format_prompt( [{"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français. Tu es spécialisé dans la mise en forme de texte pour le rendre plus lisibile et synthétique. Tu maîtrise parfaitement la grammaire et l'expression écrite, et tu es un expert en informatique. Tu n'invente jamais aucun élément, tu t'astreins à toujours reformuler exactement les phrases qu'on te fournit sans rien ajouter ni enlever à leur sens."},
    {"role": "user", "content": message}], llm._model)
    sampling_params = SamplingParams(temperature=0.7, top_p=0.8, repetition_penalty=1.05, max_tokens=len(text)+1024)
   
    start_time = time.time()  # Record the start time
    outputs = llm.generate(prompt, sampling_params)
    end_time = time.time()  # Record the end time

    generated_text = outputs[0].outputs[0].text
    tokenscount = len(outputs[0].outputs[0].token_ids)
    tokens_per_sec = tokenscount/(end_time-start_time)
    
    return generated_text,tokens_per_sec

In [9]:
llm = vllm_load(test_models["llama-3.1:w8a16"])

WARNING 09-29 21:25:25 arg_utils.py:930] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 09-29 21:25:25 config.py:1010] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 09-29 21:25:25 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16', speculative_config=None, tokenizer='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=49152, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, 

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-29 21:25:32 model_runner.py:1025] Loading model weights took 8.4927 GB
INFO 09-29 21:25:32 gpu_executor.py:122] # GPU blocks: 7096, # CPU blocks: 2048
INFO 09-29 21:25:33 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-29 21:25:33 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-29 21:25:42 model_runner.py:1456] Graph capturing finished in 10 secs.


In [10]:
instruction = """
Le texte ci-dessous est le résultat d'un transcription automatique de la voix du présentateur d'une conférence sur l'intelligence artificielle. 
Cette transcription est imparfaite : erreurs, mots incomplets, poncutation manquante, hésitations, interruptions ...
Ton travail consiste à répéter strictement le texte fourni ci-dessous, mais en corrigeant sa syntaxe et sa mise en forme :
- reformulation sous forme de phrases équivalentes mais bien contruites et sans fautes d'orthographe
- ajout de sauts de lignes de paragraphes chaque fois que le présentateur change de sujet
- génération de titres et sous-titres de chapitres au format Markdown

Voici le texte à mettre en forme :


"""

In [30]:
text, tokens_per_sec = vllm_generate(instruction, textes[0], llm)

Processed prompts: 100%|███████████| 1/1 [00:14<00:00, 14.45s/it, est. speed input: 407.78 toks/s, output: 78.50 toks/s]


In [31]:
from IPython.display import Markdown, display

display(Markdown(text))

# Introduction à l'Intelligence Artificielle

## Les Domaines de Recherche dans l'IA

Voilà, alors, sur tous ces sujets, on a vu que chaque phrase qu'on a dite là-dedans, en fait, c'est un domaine de recherche à part entière, encore aujourd'hui. Il n'y a pas de mode opératoire, de règles, etc. Donc, comment aller plus loin, comment faire de la veille ?

## La Veille et les Ressources

On a prévu au delà de cette formation tronc commun d'utiliser nos réunions innovation hebdomadaires. Robin a été intervenu pour essayer de vous montrer comment faire pour rechercher quand on sait pas. On peut commencer par YouTube, tout bêtement. Il ya plein de vidéos, plein d'explications externes. Quand vous commencez à repérer les gens qui sont intéressants, qui ont le plus de vue là-dessus, alors c'est quand même... En général, il y a une corrélation entre la popularité et la qualité des contenus.

## Les Communautés Spontanées

Il ya des communautés spontanées qui se créent en ligne. C'est pour moi extrêmement impressionnant. C'est assez incroyable. Les trucs commencent par 3 personnes qui se réunissent et à la fin ils sont plusieurs milliers. Il ya deux exemples de ça.

## Le Premier Exemple

Un exemple est la formation en début d'année, où les gens se sont dit : "Voilà, ça fait à peu près un an qu'on fait des LM en production. Qu'est-ce qu'on a appris ?" Il ya quelques personnes dans la communauté assez connues. Ils sont mis à deux ou trois au départ. Ils ont dit : "On va faire une formation où c'est un peu dans le même format qu'on fait là, avec très peu de slides et c'est juste j'essaye de restituer un peu dans l'ordre et correctement ce que j'ai appris donné un an de LM." Ils ont commencé à prévoir trois ou quatre sessions, je ne sais plus. Je me suis inscrit à ce moment-là. Et après, ça a fini en une conférence où il y a eu plus de 40 sessions et tous les grands noms du domaine sont venus chacun contribuer leur truc.

## Le Deuxième Exemple

Un autre exemple est la communauté QDA Mode, qui a démarré avec trois personnes et a fini par avoir 9 000 personnes actives. Ils ont créé un serveur Discord et une chaîne YouTube pour partager leurs connaissances et leurs expériences.

## L'Importance de la Veille

L'importance de la veille est de repérer les acteurs clés et les communautés qui se créent en ligne. Cela nous permet de nous tenir informés et de nous adapter aux évolutions du domaine.

## Les Tâches dans le Machine Learning

Dans le machine learning, il ya différentes tâches que l'on peut effectuer, comme la classification, la segmentation, la détection d'objets, etc. Il ya une nomenclature de toutes les tâches qu'on peut faire sur l'image, le texte, la voix, etc.

## Les Benchmark Académiques

Il ya des benchmark académiques qui mesurent les capacités d'un modèle sur une tâche particulière. Cela nous permet de comparer les performances des modèles et de voir comment on a amélioré les performances sur les benchmarkes au fil des années.

## L'Importance de Connaître les Tâches

Connaître les tâches est important pour faire une bonne formation en IA. Il faut être capable de distinguer les différentes tâches et de les séparer pour faire une conception de projet efficace.

## L'Apprentissage par l'Exemple

L'apprentissage par l'exemple est un principe fondamental du machine learning. On apprend par l'exemple en ajustant les paramètres d'un modèle pour minimiser l'erreur sur des exemples de données.

## L'Ajustement des Paramètres

L'ajustement des paramètres est un processus crucial dans l'apprentissage par l'exemple. On ajuste les paramètres pour minimiser l'erreur sur des exemples de données et pour trouver un modèle qui prédit correctement les résultats.

## La Construction d'un Modèle

La construction d'un modèle est un processus complexe qui implique l'ajustement des paramètres pour minimiser l'erreur sur des exemples de données. Le modèle final est imparfait, mais il est capable de prédire correctement les résultats sur de nouvelles données.

## Conclusion

En conclusion, la veille et la connaissance des tâches sont essentiels pour faire une bonne formation en IA. L'apprentissage par l'exemple et l'ajustement des paramètres sont des principes fondamentaux du machine learning. La construction d'un modèle est un processus complexe qui implique l'ajustement des paramètres pour minimiser l'erreur sur des exemples de données.

## Nvidia Nemo ASR

https://developer.nvidia.com/blog/accelerating-leaderboard-topping-asr-models-10x-with-nvidia-nemo/

https://docs.nvidia.com/nemo-framework/user-guide/latest/nemotoolkit/asr/intro.html